# Start

In [1]:
1

1

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [ ]:
import easyocr

reader = easyocr.Reader(["es"])

In [ ]:
import numpy as np

print(np.__version__)

In [ ]:
import pdfplumber

print(pdfplumber.__version__)

In [ ]:
import os

os.chdir("../")

In [ ]:
root = os.getcwd()
pdf_file = os.path.join(root, "data/raw/Factura_FE22321385375221.pdf")

In [ ]:
pdf = pdfplumber.open(pdf_file)

In [ ]:
p1 = pdf.pages[1]

In [ ]:
im = p1.to_image()
im

In [ ]:
extracted_text = p1.extract_text()
pdf.close()
extracted_text

In [ ]:
# List of words of interest in order to find the desired section
power = {
    "potencia",
    "potència",
    "poténcia",
    "Potencia",
    "Potència",
    "Poténcia",
    "potencies",
    "potencias",
    "potències",
    "poténcias",
    "Potencias",
    "Potències",
    "Poténcias",
}
maxima = {
    "maxima",
    "màxima",
    "máxima",
    "Maxima",
    "Màxima",
    "Máxima",
    "maximes",
    "maximas",
    "màximes",
    "máximas",
    "Maximas",
    "Màximes",
    "Máximas",
}
year = {"any", "año", "Any", "Año"}
words = [power, maxima, year]

In [ ]:
# Get the exact words
found = []
dummy = [
    found.append(word)
    for list_word in words
    for word in list_word
    if str(word) in extracted_text
]
found

In [ ]:
# Make the extracted text a list of str lines
extracted_text = extracted_text.splitlines()

In [ ]:
# Extract the line where the words are located
for i, line in enumerate(extracted_text):
    if found[-1] in line:
        print(f"Target words found on line {i}:")
        target_line = extracted_text[i]
        print(target_line)

In [ ]:
import re

p1 = p2 = p3 = "Unknown"
line = target_line

if "P1" in line:
    p1 = line.split("P1")[0]
    numbers = re.findall(r"\d", p1)
    # numbers.split(numbers[0])

    line = line.split("P1")[-1]
if "P2" in line:
    p2 = line.split("P2")[0]
if "P3" in line:
    line = line.split("P2")[-1]
    p3 = line.split("P3")[0]


print(f"P1: {p1}")
print(f"P2: {p2}")
print(f"P3: {p3}")

In [ ]:
import re

numbers = re.findall(r"\d", p1)
if "," in p1:
    p1 = p1.split(",")
    integers = re.findall(r"\d", p1[0])
    decimals = re.findall(r"\d", p1[-1])
    number = int(integers) + len(decimals) * 10 / int(decimals)
if "." in p1:
    p1 = p1.split(".")

In [122]:
number

NameError: name 'number' is not defined

In [127]:
int(integers)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [128]:
integers

['2']